In [5]:
import os
from ipynb.fs.defs.cml_functions import *
from time import sleep
from datetime import datetime
from linetimer import CodeTimer
import numpy as np

In [7]:
def cml_analyse(path,plot_save_path='cm_analysis/',search_radius=2):
    name = os.path.basename(os.path.splitext(path)[0])
    data = open_cm_log(path)
    # STEP 1: Rough geographic selection 
    position_list = extract_json_positions(data)
    positions_city = get_points_within_window(position_list)
    # Second round
    median = np.median(positions_city,axis=0)
    p0 = get_points_in_circle(positions_city, median, radius_km=search_radius)

    #plt.plot(median[1],median[0],'xr',ms=20)
    #plt.plot(p0[:,1],p0[:,0],'.') #plot longitude, latitude
    #mplleaflet.display()
    # STEP 2: Filter points belonging to train
    p1,D1=filter_train(p0,n_next=5,crit_thr=2)
    # STEP 3: Sort out outliers within train by taking median of 3 neighboring points
    p1_avg,D1_avg = median_with_nearest(p1,D1,n=3)
    # STEP 4: Sort points
    p2,D2 = sort_points(p1_avg,D1_avg)
    # STEP 5: Snap points to road
    match0NoLoop,match0Unique = osrm_match(p2[:,:])
    
    if plot_save_path:
        # Create result plot
        plt.plot(positions_city[:,1],positions_city[:,0],'b.')
        plt.plot(p0[:,1],p0[:,0], 'xk')
        plt.plot(p2[:,1],p2[:,0], 'ok')
        # print('With loops:',getPathLength(match0Unique))
        # print('Loops removed:',getPathLength(match0NoLoop))
        plt.plot(match0Unique[:,1],match0Unique[:,0],'b-')
        plt.plot(match0NoLoop[:,1],match0NoLoop[:,0],'r-',linewidth=3)
        #mplleaflet.display(fig=f)
        mplleaflet_update(path=plot_save_path,template='critical_length_base.html')
        #mplleaflet_update(path=os.path.join(plot_save_path,name+'.html'))
    results = dict()
    results['p0'] = p0
    results['p1'] = p1
    results['p1_avg'] = p1_avg
    results['p2'] = p2
    results['distances2'] = np.diagonal(D2,offset=1)
    results['trivial_length'] = getPathLength(p2)
    results['osrm_length'] = getPathLength(match0NoLoop)
    if len(results['distances2'])>1:
        cumsum=np.cumsum(results['distances2'])
        idx=np.argmax(cumsum>results['trivial_length']/2)
        fLarger=cumsum[idx]/results['trivial_length']-0.5
        fSmaller=0.5-cumsum[idx-1]/results['trivial_length']
        center=np.sum([results['p2'][idx-1]*fSmaller,results['p2'][idx]*fLarger],axis=0)/(fLarger+fSmaller)
    else:
        center = p2[0]
    results['trivial_center'] = center
    results['name'] = name
    results['date'] = datetime.strptime(os.path.splitext(name)[0],'%Y%m%d_%H%M%S')
    #results['osmr_length'] = getPathLength(match0NoLoop)
    return results

#cml_analyse('cm_logs/20220624_210255.txt')
#results = cml_analyse('cm_logs/20220624_203957.txt')
with CodeTimer('All'):
    results = cml_analyse('cm_logs/20220624_215158.txt',plot_save_path='test_reload/index.html')

Code block took: 21060.41540 ms
Code block 'All' took: 21104.94390 ms


ConnectionError: HTTPConnectionPool(host='192.168.178.37', port=5000): Max retries exceeded with url: /match/v1/cycling/10.111445,53.552351;10.111950,53.552898;10.112297,53.553395;10.112383,53.553572;10.112548,53.553561;10.112750,53.553618;10.112947,53.553896;10.112958,53.554062;10.113282,53.554365;10.113763,53.554954;10.113826,53.555023;10.112669,53.553731;10.116785,53.558340;10.116776,53.558368;10.117385,53.558776;10.117395,53.559133;10.117513,53.559302;10.117948,53.559734;10.118239,53.559914;10.119614,53.562273;10.119834,53.562441;10.120084,53.562739?radiuses=15;15;15;15;15;15;15;15;15;15;15;15;15;15;15;15;15;15;15;15;15;15&tidy=true&gaps=ignore (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000022BFC8AE820>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [ ]:
results_all = dict()
for ii,entry in enumerate(os.scandir('cm_logs')):
    if ii%5==0:
        if entry.path.endswith(".txt"):
            #print(entry.path)
            #try:
            result = cml_analyse(entry.path,plot_save_path='')
            #except:
            #    continue
            for k in result:
                if k not in results_all:
                    results_all[k] = []
                results_all[k].append(result[k])
            #sleep(1)
    if ii%100==0:
        print(ii)
results_all['trivial_center'] = np.array(results_all['trivial_center'])
%matplotlib widget
#plt.figure(figsize=(20,10))
plt.plot(results_all['trivial_length'],'o')

In [ ]:
%matplotlib widget
#plt.figure(figsize=(20,10))
plt.plot(results_all['date'],np.array(results_all['trivial_length']),'x')
plt.plot(results_all['date'],np.array(results_all['osrm_length']),'o')

In [ ]:
idx = 10;
print(idx,results_all['name'][idx],results_all['trivial_length'][idx])

In [ ]:
%matplotlib widget
#plt.figure(figsize=(20,10))
plt.plot(results_all['trivial_length'],'x')
plt.plot(np.array(results_all['osrm_length']),'o')

In [ ]:
f=plt.figure(figsize=(6,6))
plt.plot(results_all['trivial_center'][:,1],results_all['trivial_center'][:,0],'ok')
mplleaflet.display(fig=f)